In [1]:
import re
import numpy as np 
import pandas as pd 
import locale
import calendar

Load files

In [2]:
facts = pd.read_excel("data/homicidios.xlsx", sheet_name="HECHOS")
facts_dict = pd.read_excel("data/homicidios.xlsx", sheet_name="DICCIONARIO_HECHOS")

Treatment of data from the dataset 'facts'

In [3]:
facts

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,NaN,MOM,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,NaN,TERRADA,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,4071.0,NaN,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS


Lets show the columns info:

In [4]:
facts_dict

,variables y definiciones,descripción,Unnamed: 2
0,ID,identificador unico del siniestro,NaN
1,N_VICTIMAS,cantidad de víctimas,NaN
2,FECHA,fecha en formato dd/mm/aaaa,NaN
3,AAAA,año,NaN
4,MM,mes,NaN
5,DD,día del mes,NaN
6,HORA,hora del siniestro,NaN
7,HH,franja horaria entera,NaN
8,LUGAR_DEL_HECHO,Dirección del hecho,NaN
9,TIPO_DE_CALLE,Tipo de arteria. En el caso de intersecciones ...,NaN


Now, We need to see a summary about the facts data base

In [5]:
def data_summ(df):
    '''
    This function provides detailed information about the dtype and null values present in a dataframe
    '''

    info_dict = {'Column': [], 'Data_type': [], 'No_miss_Qty': [], '%Missing': [], 'Missing_Qty': []}
    

    for column in df.columns:
        
        info_dict['Column'].append(column)
        info_dict['Data_type'].append(df[column].apply(type).unique())
        info_dict['No_miss_Qty'].append(df[column].count())
        info_dict['%Missing'].append(round(df[column].isnull().sum() * 100 / len(df), 2))
        info_dict['Missing_Qty'].append(df[column].isnull().sum())

    df_info = pd.DataFrame(info_dict)
    
    print('\nTotal rows: ', len(df))
    print('\nTotal full null rows: ', df.isna().all(axis=1).sum())
    print('\nTotal duplicated rows: ', df.duplicated().sum())
    
    return df_info

In [6]:
data_summ(facts)


Total rows:  696

Total full null rows:  0

Total duplicated rows:  0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,ID,[<class 'str'>],696,0.00,0
1,N_VICTIMAS,[<class 'int'>],696,0.00,0
2,FECHA,[<class 'pandas._libs.tslibs.timestamps.Timest...,696,0.00,0
3,AAAA,[<class 'int'>],696,0.00,0
4,MM,[<class 'int'>],696,0.00,0
5,DD,[<class 'int'>],696,0.00,0
6,HORA,"[<class 'datetime.time'>, <class 'str'>, <clas...",696,0.00,0
7,HH,"[<class 'int'>, <class 'str'>]",696,0.00,0
8,LUGAR_DEL_HECHO,[<class 'str'>],696,0.00,0
9,TIPO_DE_CALLE,[<class 'str'>],696,0.00,0


Our data summary shows some columns in uppercase and fillies in lowercase. It also shows some columns with null data and some columns with different data types. Our first step will be to standardize column writing, parse and transform null data (if possible), and then parse columns with different data types.

## Standarize columns format

In [7]:
# Replace underscores with spaces
facts.columns = facts.columns.str.replace('_', ' ')
# The first letter is capitalized
facts.columns = facts.columns.str.title()
# Rename columns
facts = facts.rename(columns={'Id': 'Hechos_Id',                        
                                        'N Victimas': 'Número de Victimas',
                                        'Fecha': 'Fecha',
                                        'Aaaa':'Año',
                                        'Mm':'Mes',
                                        'Dd':'Dia',
                                        'Hh':'Franja Horaria',
                                        'Xy (Caba)':'XY (CABA)',
                                        'Victima': 'Víctima'})


facts.columns

Index(['Hechos_Id', 'Número de Victimas', 'Fecha', 'Año', 'Mes', 'Dia', 'Hora',
       'Franja Horaria', 'Lugar Del Hecho', 'Tipo De Calle', 'Calle', 'Altura',
       'Cruce', 'Dirección Normalizada', 'Comuna', 'XY (CABA)', 'Pos X',
       'Pos Y', 'Participantes', 'Víctima', 'Acusado'],
      dtype='object')

## Columns with null data 

- Column 'Calle' has 1 null data
- Column 'Altura' has 567 null data
- Column 'Cruce' has 171 null data
- Column 'Dirección Normalizada' has 8 null data

We can see in the table that the column 'Lugar del hecho' is related to the columns 'Calle', 'Cruce' and 'Dirección Normalizada'. We go to analize this set of columns to decide what to do with null data

In [8]:
facts[facts['Calle'].isnull()][['Dirección Normalizada', 'Cruce','Altura', 'Calle', 'Lugar Del Hecho']]

,Dirección Normalizada,Cruce,Altura,Calle,Lugar Del Hecho
119,NaN,NaN,NaN,NaN,SD


In [9]:
facts[facts['Calle'].isnull()]

,Hechos_Id,Número de Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar Del Hecho,Tipo De Calle,...,Altura,Cruce,Dirección Normalizada,Comuna,XY (CABA),Pos X,Pos Y,Participantes,Víctima,Acusado
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,...,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD


We see that the column 'Calle' has only one null value. When analyzing the entire row, we realize that this row is missing relevant information, therefore we decide to delete it

In [10]:
facts = facts.dropna(subset=['Calle']).reset_index(drop=True)

Now, we analize the column 'Altura'

In [11]:
facts[facts['Altura'].notna()][['Dirección Normalizada', 'Cruce','Altura', 'Calle', 'Lugar Del Hecho']]

,Dirección Normalizada,Cruce,Altura,Calle,Lugar Del Hecho
2,ENTRE RIOS AV. 2034,NaN,2034.0,ENTRE RIOS AV.,AV ENTRE RIOS 2034
9,ENTRE RIOS AV. 1366,NaN,1366.0,ENTRE RIOS AV.,AV ENTRE RIOS 1366
14,SUIPACHA 156,NaN,156.0,SUIPACHA,SUIPACHA 156
33,LIMA 1483,NaN,1483.0,LIMA,LIMA 1483
42,"CAMPOS, LUIS M. AV. 30",NaN,30.0,"CAMPOS, LUIS M. AV.",AV LUIS MARIA CAMPOS 30
...,...,...,...,...,...
676,TUCUMAN 3112,NaN,3112.0,TUCUMAN,TUCUMAN 3112
684,ESMERALDA 1359,NaN,1359.0,ESMERALDA,ESMERALDA 1359
686,GAONA AV. 3655,NaN,3655.0,GAONA AV.,AV. GAONA 3655
693,"PERON, EVA AV. 4071",NaN,4071.0,"PERON, EVA AV.",AV. EVA PERON 4071


Analyzing the data from the 'Altura' column, we proceed to eliminate it due to the large number of missing items (81.47%) and there being no way to fill in the columns; Investigating the data in the 'Altura' column, we have seen that it refers to the numbering of the street where the accident occurs. The dataset shows us that the majority of accidents occur at intersections or corners. In fact, this information is found in the column 'Dirección Normalizada'

In [12]:
# Drop column 'Altura'
facts= facts.drop('Altura', axis=1)

Now, we analize the column 'Cruce'

In [13]:
facts[facts['Cruce'].isnull()][['Dirección Normalizada', 'Cruce', 'Calle', 'Lugar Del Hecho']]

,Dirección Normalizada,Cruce,Calle,Lugar Del Hecho
2,ENTRE RIOS AV. 2034,NaN,ENTRE RIOS AV.,AV ENTRE RIOS 2034
9,ENTRE RIOS AV. 1366,NaN,ENTRE RIOS AV.,AV ENTRE RIOS 1366
14,SUIPACHA 156,NaN,SUIPACHA,SUIPACHA 156
33,LIMA 1483,NaN,LIMA,LIMA 1483
35,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....
...,...,...,...,...
679,AUTOPISTA 25 DE MAYO y PASCO,NaN,AUTOPISTA 25 DE MAYO,AU 25 DE MAYO KM7 (ALTURA PASCO)
684,ESMERALDA 1359,NaN,ESMERALDA,ESMERALDA 1359
686,GAONA AV. 3655,NaN,GAONA AV.,AV. GAONA 3655
693,"PERON, EVA AV. 4071",NaN,"PERON, EVA AV.",AV. EVA PERON 4071


In [14]:
facts[facts['Cruce'].notna()][['Dirección Normalizada', 'Cruce', 'Calle', 'Lugar Del Hecho']]

,Dirección Normalizada,Cruce,Calle,Lugar Del Hecho
0,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...","FERNANDEZ DE LA CRUZ, F., GRAL. AV.",PIEDRA BUENA AV.,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ
1,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV.",AV GRAL PAZ Y AV DE LOS CORRALES
3,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.","VILLEGAS, CONRADO, GRAL.",LARRAZABAL AV.,AV LARRAZABAL Y GRAL VILLEGAS CONRADO
4,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.","SAENZ PE?A, LUIS, PRES.",SAN JUAN AV.,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA
5,27 DE FEBRERO AV. y ESCALADA AV.,ESCALADA AV.,27 DE FEBRERO AV.,AV 27 DE FEBRERO Y AV ESCALADA
...,...,...,...,...
688,"BAIGORRIA y HUGO, VICTOR","HUGO, VICTOR",BAIGORRIA,BAIGORRIA Y VICTOR HUGO
689,RIVADAVIA AV. y PUEYRREDON AV.,PUEYRREDON AV.,RIVADAVIA AV.,AV. RIVADAVIA Y AV. PUEYRREDON
690,RIESTRA AV. y MOM,MOM,RIESTRA AV.,AV. RIESTRA Y MOM
691,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL.",AU DELLEPIANE Y LACARRA


We note that the 'Cruce' column refers to the intersections between two streets and has data only when the accident is referenced at the intersection of two streets. Otherwise it is null.

When the accident is at a street intersection, the data in the 'Cruce' column is found in the 'LUGAR_DEL_HECHO' column, therefore, we are going to define boolean values ​​for this column for when the accident is established at a street intersection.

In [15]:
facts['Cruce'] = facts['Cruce'].notna()

In [16]:
facts[facts['Dirección Normalizada'].isnull()][['Dirección Normalizada', 'Cruce', 'Calle', 'Lugar Del Hecho']]

,Dirección Normalizada,Cruce,Calle,Lugar Del Hecho
38,NaN,False,"LUGONES, LEOPOLDO AV.",AUTOPISTA LUGONES PK 10000
106,NaN,False,AUTOPISTA BUENOS AIRES - LA PLATA,AU BUENOS AIRES - LA PLATA KM. 4
179,NaN,False,AUTOPISTA PERITO MORENO,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6
180,NaN,False,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,AU DELLEPIANE 2400
312,NaN,False,"LUGONES, LEOPOLDO AV.",AUTOPISTA LUGONES KM 4.7
545,NaN,False,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV. KM 6,1"
620,NaN,False,AUTOPISTA BUENOS AIRES - LA PLATA,"AU BUENOS AIRES LA PLATA KM 4,5"


In [17]:
facts['Dirección Normalizada'].fillna('SD', inplace=True)

In [18]:
facts.head()

,Hechos_Id,Número de Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar Del Hecho,Tipo De Calle,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Pos X,Pos Y,Participantes,Víctima,Acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,True,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,False,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,True,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,True,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


## Columns with different data types

In [19]:
data_summ(facts)


Total rows:  695

Total full null rows:  0

Total duplicated rows:  0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,Hechos_Id,[<class 'str'>],695,0.0,0
1,Número de Victimas,[<class 'int'>],695,0.0,0
2,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,695,0.0,0
3,Año,[<class 'int'>],695,0.0,0
4,Mes,[<class 'int'>],695,0.0,0
5,Dia,[<class 'int'>],695,0.0,0
6,Hora,"[<class 'datetime.time'>, <class 'str'>, <clas...",695,0.0,0
7,Franja Horaria,"[<class 'int'>, <class 'str'>]",695,0.0,0
8,Lugar Del Hecho,[<class 'str'>],695,0.0,0
9,Tipo De Calle,[<class 'str'>],695,0.0,0


Columns 'Hora', and 'Franja Horaria'  have mixed types of data. We going to analize and tarnsform (if possible)

In [20]:
# Cantidad de valores por tipo de dato en la columna 'hora'
facts['Hora'].apply(type).value_counts()

Hora
<class 'datetime.time'>        607
<class 'str'>                   85
<class 'datetime.datetime'>      3
Name: count, dtype: int64

We going to analize columns with data type 'str'

In [21]:
datos_str = facts.loc[facts['Hora'].apply(lambda x: isinstance(x, str))]
datos_str

,Hechos_Id,Número de Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar Del Hecho,Tipo De Calle,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Pos X,Pos Y,Participantes,Víctima,Acusado
274,2018-0001,1,2018-01-01,2018,1,1,22:50:00,22,San Juan Av. Y Rincón,AVENIDA,SAN JUAN AV.,True,SAN JUAN AV. y RINCON,3,Point (106229.70400707 100696.43939679),-58.39536773,-34.62297346,PEATON-MOTO,PEATON,MOTO
275,2018-0002,1,2018-01-03,2018,1,3,16:40:00,16,Peru 937,CALLE,PERU,False,PERU 937,1,Point (108138.18410245 101140.78821350),-58.37456088,-34.61895467,BICICLETA-PASAJEROS,BICICLETA,PASAJEROS
276,2018-0003,1,2018-01-05,2018,1,5,10:12:00,10,"Pueyrredon, Honorio, Dr. Av. y Galicia",AVENIDA,"PUEYRREDON, HONORIO, DR. AV.",True,"PUEYRREDON, HONORIO, DR. AV. y GALICIA",15,Point (101348.13972450 102808.02053976),-58.44860217,-34.60395657,MULTIPLE,PEATON,MULTIPLE
277,2018-0004,1,2018-01-09,2018,1,9,09:40:00,9,Del Libertador Av. y Manzanares,AVENIDA,DEL LIBERTADOR AV.,True,DEL LIBERTADOR AV. y MANZANARES,13,Point (100183.96103971 109533.84269558),-58.46129513,-34.54332766,MULTIPLE,MOTO,MULTIPLE
278,2018-0005,1,2018-01-09,2018,1,9,22:30:00,22,Paseo Colon Av. 1019,AVENIDA,PASEO COLON AV.,False,PASEO COLON AV. 1019,1,Point (108641.50518777 101075.93727953),-58.36907201,-34.61953517,PEATON-PASAJEROS,PEATON,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,2018-0138,1,2018-12-21,2018,12,21,09:30:00,9,Bermudez 1893,CALLE,BERMUDEZ,False,BERMUDEZ 1893,10,Point (95944.83421152 100945.68379559),-58.50751833,-34.62073702,PASAJEROS-PASAJEROS,PASAJEROS,PASAJEROS
412,2018-0139,1,2018-12-22,2018,12,22,06:30:00,6,Santander 1974,CALLE,SANTANDER,False,SANTANDER 1974,7,Point (101510.09611838 98933.71958961),-58.44682986,-34.63888084,MOTO-MOTO,MOTO,MOTO
415,2018-0142,1,2018-12-25,2018,12,25,13:30:00,13,"PAZ, GRAL. AV. 5172",GRAL PAZ,"PAZ, GRAL. AV.",False,"PAZ, GRAL. AV. 5172",12,Point (95716.93710226 106295.47905158),-58.50997604,-34.57251072,CARGAS-PASAJEROS,CARGAS,PASAJEROS
416,2018-0143,1,2018-12-29,2018,12,29,05:25:00,5,"Olivera Av. y Alberdi, Juan Bautista Av.",AVENIDA,OLIVERA AV.,True,"OLIVERA AV. y ALBERDI, JUAN BAUTISTA AV.",10,Point (97912.79273192 98756.38729095),-58.48606475,-34.64047818,MOTO-AUTO,MOTO,AUTO


We see that the str type data has a time format, except one that is SD. We will convert the data to time type and perform an imputation with the SD data with reference to the mode

In [22]:
facts['Hora'] = pd.to_datetime(facts['Hora'], format='%H:%M:%S', errors='coerce').dt.time

In [23]:
facts['Hora'].apply(type).value_counts()

Hora
<class 'datetime.time'>                          694
<class 'pandas._libs.tslibs.nattype.NaTType'>      1
Name: count, dtype: int64

In [24]:
# calculate the mode
hora_moda = facts['Hora'].mode().iloc[0]
# The value None is replaced by the most common time
facts['Hora'].fillna(hora_moda, inplace=True)
facts.loc[517]

Hechos_Id                                             2019-0103
Número de Victimas                                            1
Fecha                                       2019-12-18 00:00:00
Año                                                        2019
Mes                                                          12
Dia                                                          18
Hora                                                   09:00:00
Franja Horaria                                               SD
Lugar Del Hecho                         PAZ, GRAL. AV. Y GRIVEO
Tipo De Calle                                          GRAL PAZ
Calle                                            PAZ, GRAL. AV.
Cruce                                                      True
Dirección Normalizada                   PAZ, GRAL. AV. y GRIVEO
Comuna                                                       11
XY (CABA)                Point (94643.11254058 103831.57115061)
Pos X                                   

In [25]:
datos_str = facts.loc[facts['Franja Horaria'].apply(lambda x: isinstance(x, str))]
datos_str

,Hechos_Id,Número de Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar Del Hecho,Tipo De Calle,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Pos X,Pos Y,Participantes,Víctima,Acusado
517,2019-0103,1,2019-12-18,2019,12,18,09:00:00,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


In [26]:
# Change de str data by int data. 
facts['Franja Horaria'] = facts['Franja Horaria'].apply(lambda x: int(hora_moda.hour) if x == "SD" else x)
# Verify
facts['Franja Horaria'].apply(type).value_counts()

Franja Horaria
<class 'int'>    695
Name: count, dtype: int64

In [27]:
data_summ(facts)


Total rows:  695

Total full null rows:  0

Total duplicated rows:  0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,Hechos_Id,[<class 'str'>],695,0.0,0
1,Número de Victimas,[<class 'int'>],695,0.0,0
2,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,695,0.0,0
3,Año,[<class 'int'>],695,0.0,0
4,Mes,[<class 'int'>],695,0.0,0
5,Dia,[<class 'int'>],695,0.0,0
6,Hora,[<class 'datetime.time'>],695,0.0,0
7,Franja Horaria,[<class 'int'>],695,0.0,0
8,Lugar Del Hecho,[<class 'str'>],695,0.0,0
9,Tipo De Calle,[<class 'str'>],695,0.0,0


## Geographic Data
Now let's analyze the columns corresponding to the geographical location of the accidents:

In [28]:
unique_values_X = facts['Pos X'].unique()

# Show unique values
print(unique_values_X)

['-58.47533969' '-58.50877521' '-58.39040293' '-58.46503904'
 '-58.38718297' '-58.44451316' '-58.50095869' '-58.38048577'
 '-58.39114932' '-58.43760020' '-58.36882665' '-58.44989671'
 '-58.50990884' '-58.37952223' '-58.45157703' '-58.38144696'
 '-58.46244952' '-58.49216191' '-58.44497874' '-58.38199706'
 '-58.47378664' '-58.47064556' '-58.40623949' '-58.39228965'
 '-58.37813396' '-58.46275892' '-58.49683458' '-58.40603217'
 '-58.46076550' '-58.47832463' '-58.43213235' '-58.46743471'
 '-58.38247061' '-58.44759966' '-58.37714647568196' '-58.47305463' '.'
 '-58.48429180' '-58.48695354' '-58.40021959' '-58.42957885'
 '-58.38247542' '-58.38222219' '-58.49485242' '-58.45227946'
 '-58.48455411' '-58.39056593' '-58.39118948' '-58.49021952'
 '-58.49896559' '-58.47461076' '-58.46210891' '-58.37027770'
 '-58.40491455' '-58.45504154' '-58.52927982' '-58.48721060'
 '-58.42273092' '-58.44788671' '-58.38110563' '-58.42378834'
 '-58.48727942' '-58.37864583' '-58.50151750' '-58.43233425'
 '-58.41971993

In [29]:
facts[facts['Pos X'] == '.']

,Hechos_Id,Número de Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar Del Hecho,Tipo De Calle,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Pos X,Pos Y,Participantes,Víctima,Acusado
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,13,Point (. .),.,.,MOTO-SD,MOTO,SD
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,False,SD,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
138,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,AUTOPISTA 25 DE MAYO,False,AUTOPISTA 25 DE MAYO,0,Point (. .),.,.,SD-SD,SD,SD
175,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.",False,"LUGONES, LEOPOLDO AV.",14,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
179,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,False,SD,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
180,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,False,SD,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO
255,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,False,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),.,.,MOTO-PASAJEROS,MOTO,PASAJEROS
312,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,14,Point (. .),.,.,PEATON-AUTO,PEATON,AUTO
545,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,14,Point (. .),.,.,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
558,2020-0039,1,2020-09-01,2020,9,1,19:17:42,19,MURGUIONDO 2700,CALLE,MURGUIONDO,False,MURGUIONDO,9,Point (. .),.,.,PEATON-CARGAS,PEATON,CARGAS


In [30]:
facts[facts['XY (CABA)'] == 'Point (. .)']


,Hechos_Id,Número de Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar Del Hecho,Tipo De Calle,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Pos X,Pos Y,Participantes,Víctima,Acusado
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,False,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,SD,SD
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,13,Point (. .),.,.,MOTO-SD,MOTO,SD
71,2016-0096,1,2016-07-25,2016,7,25,07:00:00,7,"AUTOPISTA DELLEPIANE LUIS TTE. GRAL. KM. 2,3",AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,False,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,8,Point (. .),-58.47433193007387,-34.66684950051973,MOTO-CARGAS,MOTO,CARGAS
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,False,SD,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
138,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,AUTOPISTA 25 DE MAYO,False,AUTOPISTA 25 DE MAYO,0,Point (. .),.,.,SD-SD,SD,SD
175,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.",False,"LUGONES, LEOPOLDO AV.",14,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
179,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,False,SD,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
180,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,False,SD,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO
255,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,False,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),.,.,MOTO-PASAJEROS,MOTO,PASAJEROS
312,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,14,Point (. .),.,.,PEATON-AUTO,PEATON,AUTO


We find 14 rows that do not present the coordinates and we also see 10 records that do not present the 'Pos X' and 'Pos X' data either. To continue with the analysis, 0 will be imputed in such a way that it serves to exclude the value in the dashboard to be carried out.

In [31]:
facts[['Pos X', 'Pos Y']] = facts[['Pos X', 'Pos Y']].replace('.', 0)
facts['XY (CABA)'] = facts['XY (CABA)'].replace('Point (. .)', 0)
facts[facts['Pos X'] == 0]

,Hechos_Id,Número de Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar Del Hecho,Tipo De Calle,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Pos X,Pos Y,Participantes,Víctima,Acusado
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,13,0,0,0,MOTO-SD,MOTO,SD
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,False,SD,4,0,0,0,MOTO-CARGAS,MOTO,CARGAS
138,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,AUTOPISTA 25 DE MAYO,False,AUTOPISTA 25 DE MAYO,0,0,0,0,SD-SD,SD,SD
175,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.",False,"LUGONES, LEOPOLDO AV.",14,0,0,0,MOTO-CARGAS,MOTO,CARGAS
179,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,False,SD,9,0,0,0,MOTO-CARGAS,MOTO,CARGAS
180,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,False,SD,7,0,0,0,AUTO-AUTO,AUTO,AUTO
255,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,False,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,0,0,0,MOTO-PASAJEROS,MOTO,PASAJEROS
312,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,14,0,0,0,PEATON-AUTO,PEATON,AUTO
545,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,14,0,0,0,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
558,2020-0039,1,2020-09-01,2020,9,1,19:17:42,19,MURGUIONDO 2700,CALLE,MURGUIONDO,False,MURGUIONDO,9,0,0,0,PEATON-CARGAS,PEATON,CARGAS


## Involved in the facts

Now, Finally we will analyze the columns 'Participantes', 'Víctima' and 'Acusado'

In [32]:
facts['Participantes'].unique()

array(['MOTO-AUTO', 'AUTO-PASAJEROS', 'MOTO-SD', 'MOTO-PASAJEROS',
       'MOTO-OBJETO FIJO', 'MOTO-CARGAS', 'PEATON-AUTO', 'PEATON-MOTO',
       'PEATON-PASAJEROS', 'AUTO-AUTO', 'AUTO-OBJETO FIJO',
       'PEATON-CARGAS', 'MOTO-MOTO', 'MULTIPLE', 'SD-SD', 'AUTO-CARGAS',
       'AUTO-SD', 'MOTO-MOVIL', 'CARGAS-CARGAS', 'PEATON-SD',
       'BICICLETA-CARGAS', 'SD-CARGAS', 'BICICLETA-PASAJEROS',
       'PASAJEROS-PASAJEROS', 'MOVIL-PASAJEROS', 'CARGAS-OBJETO FIJO',
       'PEATON-BICICLETA', 'SD-AUTO', 'CARGAS-AUTO', 'BICICLETA-AUTO',
       'MOTO-BICICLETA', 'SD-MOTO', 'MOVIL-CARGAS', 'PASAJEROS-AUTO',
       'AUTO-MOVIL', 'CARGAS-PASAJEROS', 'BICICLETA-TREN',
       'BICICLETA-OTRO', 'MOTO-OTRO', 'PEATON_MOTO-MOTO', 'PASAJEROS-SD'],
      dtype=object)

In [33]:
facts['Víctima'].unique()

array(['MOTO', 'AUTO', 'PEATON', 'SD', 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL', 'OBJETO FIJO', 'PEATON_MOTO'], dtype=object)

In [34]:
facts['Acusado'].unique()

array(['AUTO', 'PASAJEROS', 'SD', 'OBJETO FIJO', 'CARGAS', 'MOTO',
       'MULTIPLE', 'OTRO', 'BICICLETA', 'TREN'], dtype=object)

The column 'Víctima' contains the values 'OBJETO FIJO' and 'PEATON_MOTO' that are not described in the data dictionary. We proceed to observe these values.

In [35]:
facts[facts['Víctima'].isin(['OBJETO FIJO', 'PEATON_MOTO'])]

,Hechos_Id,Número de Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar Del Hecho,Tipo De Calle,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Pos X,Pos Y,Participantes,Víctima,Acusado
229,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OBJETO FIJO,AUTO
582,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,NUEVA YORK Y ALTA GRACIA,CALLE,NUEVA YORK,True,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,PEATON_MOTO-MOTO,PEATON_MOTO,MOTO


We see that in the first case the values ​​of the  columns 'Víctima' and 'Acusado' are inverted. On the other hand, a small error is possible in the column 'Participantes' with the value 'PEATON_MOTO'. We proceed to correct them:

In [36]:
# Invertimos los valores de las columnas 'VICTIMA' y 'ACUSADO' en los índices 237 y 238
facts.loc[[229], ['Víctima', 'Acusado']] = facts.loc[[229], ['Acusado', 'Víctima']].values

# Reemplazamos los valores de las columnas 'PARTICIPANTES' y 'VICTIMA' en los índices 602 y 603
facts.loc[[582], ['Participantes', 'Víctima']] = facts.loc[[582], ['Participantes', 'Víctima']].replace({'PEATON_MOTO-MOTO': 'PEATON-MOTO', 'PEATON_MOTO': 'PEATON'})
facts[facts['Hechos_Id'].isin(['2017-0108', '2020-0063'])]

,Hechos_Id,Número de Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar Del Hecho,Tipo De Calle,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Pos X,Pos Y,Participantes,Víctima,Acusado
229,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,AUTO,OBJETO FIJO
582,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,NUEVA YORK Y ALTA GRACIA,CALLE,NUEVA YORK,True,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,PEATON-MOTO,PEATON,MOTO


In [37]:
locale.setlocale(locale.LC_TIME, 'es_ES')

# Agregar una nueva columna 'Día semana' con nombres de días en español
facts['Día semana'] = facts['Fecha'].dt.strftime('%A').str.capitalize()

facts['Mes'] = facts['Mes'].apply(lambda x: calendar.month_name[x]).str.capitalize()

# Restaurar la configuración regional a la predeterminada (en inglés)
locale.setlocale(locale.LC_TIME, 'C')

# Mostrar el DataFrame resultante
facts.head()

,Hechos_Id,Número de Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar Del Hecho,Tipo De Calle,...,Cruce,Dirección Normalizada,Comuna,XY (CABA),Pos X,Pos Y,Participantes,Víctima,Acusado,Día semana
0,2016-0001,1,2016-01-01,2016,Enero,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,True,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,Viernes
1,2016-0002,1,2016-01-02,2016,Enero,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,True,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado
2,2016-0003,1,2016-01-03,2016,Enero,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,False,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,Domingo
3,2016-0004,1,2016-01-10,2016,Enero,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,True,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD,Domingo
4,2016-0005,1,2016-01-21,2016,Enero,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,True,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS,Jueves


In [38]:
def get_season(mes):
    if mes in [12, 1, 2]:
        return 'Verano'
    elif mes in [3, 4, 5]:
        return 'Otoño'
    elif mes in [6, 7, 8]:
        return 'Invierno'
    elif mes in [9, 10, 11]:
        return 'Primavera'
    else:
        return 'Desconocido'

# Agregar una nueva columna 'Estación'
facts['Estación'] = facts['Fecha'].dt.month.apply(get_season)
facts

,Hechos_Id,Número de Victimas,Fecha,Año,Mes,Dia,Hora,Franja Horaria,Lugar Del Hecho,Tipo De Calle,...,Dirección Normalizada,Comuna,XY (CABA),Pos X,Pos Y,Participantes,Víctima,Acusado,Día semana,Estación
0,2016-0001,1,2016-01-01,2016,Enero,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,Viernes,Verano
1,2016-0002,1,2016-01-02,2016,Enero,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado,Verano
2,2016-0003,1,2016-01-03,2016,Enero,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,Domingo,Verano
3,2016-0004,1,2016-01-10,2016,Enero,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD,Domingo,Verano
4,2016-0005,1,2016-01-21,2016,Enero,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS,Jueves,Verano
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,2021-0093,1,2021-12-13,2021,Diciembre,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO,Lunes,Verano
691,2021-0094,1,2021-12-20,2021,Diciembre,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO,Lunes,Verano
692,2021-0095,1,2021-12-30,2021,Diciembre,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS,Jueves,Verano
693,2021-0096,1,2021-12-15,2021,Diciembre,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS,Miércoles,Verano


Finally, we reorder de columns

In [39]:
facts.columns

Index(['Hechos_Id', 'Número de Victimas', 'Fecha', 'Año', 'Mes', 'Dia', 'Hora',
       'Franja Horaria', 'Lugar Del Hecho', 'Tipo De Calle', 'Calle', 'Cruce',
       'Dirección Normalizada', 'Comuna', 'XY (CABA)', 'Pos X', 'Pos Y',
       'Participantes', 'Víctima', 'Acusado', 'Día semana', 'Estación'],
      dtype='object')

In [40]:
new_order = [
    'Hechos_Id', 'Número de Victimas', 'Fecha', 'Año', 'Mes', 'Dia', 'Día semana', 'Estación',
    'Hora', 'Franja Horaria', 'Lugar Del Hecho', 'Tipo De Calle', 'Calle', 'Cruce', 'Dirección Normalizada',
    'Comuna', 'XY (CABA)', 'Pos X', 'Pos Y', 'Participantes', 'Víctima', 'Acusado'
]

# Reorganizar las columnas del DataFrame
facts = facts[new_order]
facts

,Hechos_Id,Número de Victimas,Fecha,Año,Mes,Dia,Día semana,Estación,Hora,Franja Horaria,...,Calle,Cruce,Dirección Normalizada,Comuna,XY (CABA),Pos X,Pos Y,Participantes,Víctima,Acusado
0,2016-0001,1,2016-01-01,2016,Enero,1,Viernes,Verano,04:00:00,4,...,PIEDRA BUENA AV.,True,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,Enero,2,Sábado,Verano,01:15:00,1,...,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,Enero,3,Domingo,Verano,07:00:00,7,...,ENTRE RIOS AV.,False,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,Enero,10,Domingo,Verano,00:00:00,0,...,LARRAZABAL AV.,True,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,Enero,21,Jueves,Verano,05:20:00,5,...,SAN JUAN AV.,True,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,2021-0093,1,2021-12-13,2021,Diciembre,13,Lunes,Verano,17:10:00,17,...,RIESTRA AV.,True,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
691,2021-0094,1,2021-12-20,2021,Diciembre,20,Lunes,Verano,01:10:00,1,...,"DELLEPIANE, LUIS, TTE. GRAL.",True,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
692,2021-0095,1,2021-12-30,2021,Diciembre,30,Jueves,Verano,00:43:00,0,...,GAONA AV.,True,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS
693,2021-0096,1,2021-12-15,2021,Diciembre,15,Miércoles,Verano,10:30:00,10,...,"PERON, EVA AV.",False,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS


Save clean dataframe in a CSV file

In [41]:
archivo = f'data/csv/facts_clean.csv'
facts.to_csv(archivo, index=False, encoding='utf-8')
     